In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train_V2.csv')
test = pd.read_csv('../input/test_V2.csv')

In [ ]:
train.head(10)

In [ ]:
train.isnull().sum()

Drop invalid data

In [ ]:
train = train.dropna()

In [ ]:
train['FinalPlace'] = (train['numGroups']-1)*(1-train['winPlacePerc'])+1
df = train.drop(['Id','groupId','matchId', 'killPoints','maxPlace','numGroups','matchType','matchDuration', 'rankPoints', 'winPoints'],axis=1)
df.head(10)

In [ ]:
train.plot(x="kills",y="winPlacePerc", kind="scatter",figsize = (10,7))

We can see that players who kill lots of other players usually have better placemnet in the end

In [ ]:
train.plot(x="walkDistance",y="winPlacePerc", kind="scatter",figsize = (10,7))

In [ ]:
train['numsofplayers'] = train.groupby('matchId')['matchId'].transform('count')
plt.figure(figsize=(15,10))
sns.countplot(train[train['numsofplayers']>=80]['numsofplayers'])
plt.show()

most matches have around 96 players

We also can see that players wiht the longest walking distance win much more games 

In [ ]:
import seaborn as sns
corr = df.corr()
plt.figure(figsize=(16, 16))
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values, annot = True)

In [ ]:
cordf = corr['winPlacePerc']
cordf=cordf.drop(['winPlacePerc'])
cordf

In [ ]:
W,V=np.linalg.eig(corr)
W

Check Collinearity. We observed that the value of index 2 and 3 are close to 0.  

In [ ]:
V[:,2]

We observe that values of index 2, 6, 7, 8, 18 are relatively large. 

In [ ]:
V[:,3]

From V[:,3], we found that values of index 2, 6, 7, 8, 18 are not close to 0. 

Therefore, index 2, 6, 7, 8, 18 have strong collinearity, which are 'damageDealt', 'killPlace', 'kills' ,'killStreaks', 'weaponsAcquired'. 
Take attribute 'kills' and drop others

In [ ]:
df2 = df.drop(['damageDealt', 'killPlace', 'killStreaks','weaponsAcquired'],axis=1)
df2.head(5)

In [ ]:
corr2 = df2.corr()
cordf2 = corr2['winPlacePerc']
cordf2=cordf2.drop(['winPlacePerc', 'FinalPlace'])
cordf2

For correlation in range 0.2 to 0.4, we consider as moderate correlation. Therefore, we take attributes whose correlation is greater than 0.2 as coefficient

In [ ]:
coeff = corr2[corr2['winPlacePerc'].values>0.2]
coeff = coeff['winPlacePerc'].drop(['winPlacePerc'])
coeff

In [ ]:
kill_assist = train[['assists','kills','winPlacePerc']]
kill_assist['Kill_assist'] = train['assists'] + train['kills']
kill_assist.corr()


In [ ]:
distance = train[['walkDistance','rideDistance','winPlacePerc']]
distance['distance_total'] = distance['rideDistance'] + distance['walkDistance']
distance.corr()

Adding distance_total ends up having worse performance so it was not used in our data trainning  

In [ ]:
items = train [['boosts','heals','winPlacePerc']]
items['item_use'] = items['boosts']+ items['heals']
items.corr()


In [ ]:
train['item_use'] = train['heals'] + train['boosts']
test['item_use'] = test['heals'] + test['boosts']

In [ ]:
train['Kill_assist'] = train['kills'] + train['assists']
test['Kill_assist'] = test['kills'] + test['assists']

In [ ]:
train['distance_total'] = train['rideDistance'] + train['walkDistance']
test['distance_total'] = test['rideDistance'] + test['walkDistance']


For players that travel 0 distance but have kills, we consider them as cheaters. Also, if players kill more than 15 enemies in a game, they were matched to a game that is significantly unfair or they were cheating. Finally, for players who kills more than 5 people in a game but all are headshots, we consider then as cheaters. 

In [ ]:
train['cheaters'] = (((train['kills'] > 0) & (train['distance_total'] == 0))|(train['kills'] > 15)|((train['kills']>5)&(train['kills']==train['headshotKills'])))
train[train['cheaters'] == True].head(10)

those players are cheates, and they are useless in our data training

In [ ]:
train['cheaters'].sum()

In [ ]:
train.drop(train[train['cheaters'] == True].index, inplace=True)


In [ ]:
df_sample = train.sample(100000)
df_sample.head(4)

In [ ]:
xs_train = df_sample[[ 'DBNOs', 'Kill_assist', 'longestKill', 'item_use', 'rideDistance','walkDistance']]
ys_train = df_sample['winPlacePerc']
xs_test = test[[ 'DBNOs','Kill_assist', 'longestKill','item_use', 'rideDistance','walkDistance']]

We test our result on a sample dataset, so it wont take too long to run

In [ ]:
x_train = train[[ 'DBNOs', 'Kill_assist', 'longestKill', 'item_use','revives', 'rideDistance','walkDistance']]
y_train = train['winPlacePerc']
x_test = test[[ 'DBNOs','Kill_assist', 'longestKill','item_use','revives', 'rideDistance','walkDistance']]
test_id = test.Id



In [ ]:
from sklearn import linear_model

ols = linear_model.LinearRegression()
model = ols.fit(xs_train, ys_train)
output = model.predict(xs_test)
ols.score(xs_train, ys_train)
#print(model.coef_)

* n_estimators =The number of trees in the forest.
* max_depth =The maximum depth of the tree

In [ ]:
model = RandomForestRegressor(n_estimators=1,max_depth =40)

model.fit(xs_train,ys_train)
output = model.predict(xs_test)
model.score(xs_train,ys_train)

 the higher the number of n_estimators the better to learn the data but it will cause much slower performance

In [ ]:
model = RandomForestRegressor(n_estimators=10,max_depth =40)
model.fit(xs_train,ys_train)
output = model.predict(xs_test)
model.score(xs_train,ys_train)

In [ ]:
model = RandomForestRegressor(n_estimators=20,max_depth =40)
model.fit(xs_train,ys_train)
output = model.predict(xs_test)
model.score(xs_train,ys_train)

In [ ]:
model.feature_importances_

walkDistance is the most important feature

In [ ]:
from sklearn.neighbors import KNeighborsRegressor  
model = KNeighborsRegressor(n_neighbors =2) #Algorithm used to compute the nearest neighbors
model.fit(xs_train,ys_train)
output = model.predict(xs_test)
model.score(xs_train,ys_train)

In [ ]:
model = KNeighborsRegressor(n_neighbors =5)
model.fit(xs_train,ys_train)
output = model.predict(xs_test)
model.score(xs_train,ys_train)

In [ ]:
model = KNeighborsRegressor(n_neighbors =7)
model.fit(xs_train,ys_train)
output = model.predict(xs_test)
model.score(xs_train,ys_train)

In [ ]:
xs_train.head(5)

In [ ]:
xs_train.dtypes

Random forest has the highest score so we use it for our trainning model

In [ ]:
# model = RandomForestRegressor(n_estimators=20,max_depth =40)
# model.fit(x_train,y_train)
# output = model.predict(x_test)
# model.score(x_train,y_train)

In [ ]:
# submission=pd.DataFrame({'Id':test_id,'winPlacePerc':output})
# submission.to_csv('Joe1_rf_submission.csv', index=False)

In [ ]:
import lightgbm as lgb

model = lgb.LGBMRegressor(objective='regression')
model.fit(x_train,y_train)
output = model.predict(x_test)
model.score(x_train,y_train)


In [ ]:
model = RandomForestRegressor(n_estimators=20,max_depth =40)
model.fit(x_train,y_train)
output = model.predict(x_test)
model.score(x_train,y_train)

In [ ]:
submission=pd.DataFrame({'Id':test_id,'winPlacePerc':output})
submission.to_csv('Joe1_rf_submission.csv', index=False)